In [181]:
from qiskit import ClassicalRegister, QuantumCircuit, QuantumRegister
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt
import bitstring
from qiskit_aer import AerSimulator
from qiskit.transpiler.passes.synthesis import SolovayKitaev
from qiskit.synthesis import generate_basic_approximations
from qiskit.quantum_info import Operator

from qiskit_aer.noise import (NoiseModel, QuantumError, ReadoutError,
    pauli_error, depolarizing_error, thermal_relaxation_error)

from qiskit.circuit.library import UnitaryGate

In [182]:
qr = QuantumRegister(15,"q")
cbits = ClassicalRegister(7,"c")

qc = QuantumCircuit(qr,cbits)

qc.h(0), qc.h(1), qc.h(2), qc.h(7), qc.h(11), qc.h(12)

qc.cx(12,3), qc.cx(7,13), qc.cx(11,14)

qc.barrier()

qc.cx(0,3), qc.cx(7,4), qc.cx(11,8)
qc.cx(1,3), qc.cx(7,5), qc.cx(11,9)
qc.cx(2,3), qc.cx(7,6), qc.cx(11,10)

qc.cx(12,3), qc.cx(7,13), qc.cx(11,14)

qc.h(12)
qc.measure(12,0)
qc.measure(13,1)
qc.measure(14,2)

qc.barrier()
for i in range(8):
    qc.cx(i,i+4)

qc.barrier()

qc.reset(12), qc.h(12)
qc.reset(13)

qc.cx(12,4), qc.cx(7,13), qc.cx(12,7), qc.cx(4,13), qc.cx(12,5), qc.cx(6,13), qc.cx(12,6), qc.cx(5,13)

qc.measure(12,3), qc.measure(13,4)

qc.barrier()

qc.reset(12), qc.h(12)
qc.reset(13)

qc.cx(12,13), qc.cx(8,13), qc.cx(10,12), qc.cx(9,12), qc.cx(11,13), qc.cx(12,13)
qc.h(12)
qc.measure(12,5), qc.measure(13,6)


qc.draw('latex_source', justify="left", initial_state=True, filename="carbon.tex")#, filename="test.tex")

'\\documentclass[border=2px]{standalone}\n\n\\usepackage[braket, qm]{qcircuit}\n\\usepackage{graphicx}\n\n\\begin{document}\n\\scalebox{1.0}{\n\\Qcircuit @C=1.0em @R=0.2em @!R { \\\\\n\t \t\\nghost{{q}_{0} : \\ket{{0}} } & \\lstick{{q}_{0} : \\ket{{0}} } & \\gate{\\mathrm{H}} & \\qw & \\qw & \\qw \\barrier[0em]{14} & \\qw & \\ctrl{3} & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw \\barrier[0em]{14} & \\qw & \\ctrl{4} & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw \\barrier[0em]{14} & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw \\barrier[0em]{14} & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw\\\\\n\t \t\\nghost{{q}_{1} : \\ket{{0}} } & \\lstick{{q}_{1} : \\ket{{0}} } & \\gate{\\mathrm{H}} & \\qw & \\qw & \\qw & \\qw & \\qw & \\ctrl{2} & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\ctrl{4} & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\qw & \\

In [183]:
sqrt_t = ([[1,0],[0,np.exp((np.pi/8)*1j)]])
cool = UnitaryGate(sqrt_t)

In [ ]:
qc = QuantumCircuit(1)

qc.h(0)
for i in range(8):
    qc.append(cool, [0])
qc.h(0)

qc.measure_all()

sim = AerSimulator()
    
job = sim.run(qc, shots=100)

result = job.result()
counts = result.get_counts()

print(counts)
qc.draw("mpl")

In [ ]:
qc = QuantumCircuit(1)

#qc.rz(+np.pi/16, 0)
qc.rz(2*np.pi*0.05, 0)

print("Original circuit:")
print(qc.draw())
 
basis = ["tdg", "t", "h","z","s","sdg"]
approx = generate_basic_approximations(basis, depth=2)
skd = SolovayKitaev(recursion_degree=1)#, basic_approximations=approx)
 
discretized = skd(qc)
 
print("Discretized circuit:")
print(discretized.draw())
 
print("Error:", np.linalg.norm(Operator(qc).data - Operator(discretized).data))

#discretized.draw("latex")

Original circuit:
   ┌──────────┐
q: ┤ Rz(π/10) ├
   └──────────┘
Discretized circuit:
global phase: π/8
   ┌───┐┌─────┐┌───┐┌─────┐┌───┐┌─────┐┌───┐┌─────┐┌───┐┌─────┐┌─────┐┌───┐»
q: ┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ Tdg ├┤ H ├»
   └───┘└─────┘└───┘└─────┘└───┘└─────┘└───┘└─────┘└───┘└─────┘└─────┘└───┘»
«   ┌───┐┌───┐┌───┐┌───┐┌─────┐┌───┐┌─────┐┌───┐┌─────┐┌───┐┌─────┐┌───┐┌─────┐»
«q: ┤ T ├┤ H ├┤ T ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ Tdg ├»
«   └───┘└───┘└───┘└───┘└─────┘└───┘└─────┘└───┘└─────┘└───┘└─────┘└───┘└─────┘»
«   ┌─────┐┌───┐┌─────┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐┌───┐»
«q: ┤ Tdg ├┤ H ├┤ Tdg ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├┤ T ├┤ H ├»
«   └─────┘└───┘└─────┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘└───┘»
«   ┌───┐
«q: ┤ T ├
«   └───┘
Error: 0.08731790370618465


In [ ]:
hmm = discretized.data
h = 0
t = 0
decomp = []
for instruction in hmm:
    if instruction.name == "h":
        h+=1
        decomp.append("h")
    if instruction.name == "t":
        t+=1
        decomp.append("t")
    if instruction.name == "tdg":
        t+=1
        decomp.append("tdg")
    #print(instruction.name)
print(h,t)
print(decomp)
for i in range(len(decomp)-1):
    if decomp[i] == decomp[i+1]:
        if decomp[i] == "t":
            decomp[i+1] = "s"
            decomp [i] = 0
        if decomp[i] == "tdg":
            decomp[i+1] ="sdg"
            decomp [i] = 0
decomp.remove(0)
print(decomp.count("t")+decomp.count("tdg"))
print(decomp)

19 21
['h', 'tdg', 'h', 'tdg', 'h', 'tdg', 'h', 'tdg', 'h', 'tdg', 'tdg', 'h', 't', 'h', 't', 'h', 'tdg', 'h', 'tdg', 'h', 'tdg', 'h', 'tdg', 'h', 'tdg', 'tdg', 'h', 'tdg', 'h', 't', 'h', 't', 'h', 't', 'h', 't', 'h', 't', 'h', 't']
17
['h', 'tdg', 'h', 'tdg', 'h', 'tdg', 'h', 'tdg', 'h', 'sdg', 'h', 't', 'h', 't', 'h', 'tdg', 'h', 'tdg', 'h', 'tdg', 'h', 'tdg', 'h', 0, 'sdg', 'h', 'tdg', 'h', 't', 'h', 't', 'h', 't', 'h', 't', 'h', 't', 'h', 't']


In [ ]:
unit = np.array([[1,0],[0,1]])

t = np.array([[1,0],[0,np.exp((np.pi/4)*1j)]])

tdg = np.array([[1,0],[0,np.exp(-(np.pi/4)*1j)]])

s = np.array([[1,0],[0,np.exp((np.pi/2)*1j)]])

sdg = np.array([[1,0],[0,np.exp(-(np.pi/2)*1j)]])

h = np.array([[1,1],[1,-1]])
h = (1/np.sqrt(2))*h

test = np.array([[1,0],[0,np.exp((2*np.pi*0.05)*1j)]])

#print(test)

ok = np.matmul(unit,unit)

for instruction in hmm:
    if instruction.name == "t":
        ok = np.matmul(ok, t)
    if instruction.name == "tdg":
        ok = np.matmul(ok, tdg)
    if instruction.name == "h":
        ok = np.matmul(ok, h)

In [ ]:
def dist(U,approx):
    ehm = np.conjugate(U.T)
    next = np.matmul(ehm, approx)
    hmm = np.abs(np.trace(next))
    return np.sqrt((2-hmm)/2)

def T(a,b):
    b = b*np.exp(1j*(np.pi/4))
    return a,b

def A(a,b):                             # = T adjoint
    b = b*np.exp(-1j*(np.pi/4))
    return a,b

def S(a,b):                             
    b = b*np.exp(1j*(np.pi/2))
    return a,b

def B(a,b):                             # = S adjoint
    b = b*np.exp(-1j*(np.pi/2))
    return a,b

def H(a,b):
    a = (1/(2**0.5))*(a+b)
    b = (1/(2**0.5))*(a-b)
    return a,b

def dustin(k: int, tgates: int):
    combs = ["A","T", "S", "B"]
    val_a , val_b = [],[]
    for i in range(tgates-1):
        all1 = [i+"A" for i in combs]
        all2 = [i+"T" for i in combs]
        all3 = [i+"S" for i in combs]
        all4 = [i+"B" for i in combs]
        combs = all1 + all2 + all3 + all4
    for i in range(len(combs)):
        seq = combs[i]
        a = 0
        b = 1
        for i in seq:
            if i == "A":
                a,b = H(a,b)
                a,b = A(a,b)
            if i == "T":
                a,b = H(a,b)
                a,b = T(a,b)
            if i == "S":
                a,b = H(a,b)
                a,b = S(a,b)
            if i == "B":
                a,b = H(a,b)
                a,b = B(a,b)
        val_a.append(a)
        val_b.append(b)
    angle = np.exp((1j*2*np.pi)/(2**(k+3)))
    winner = 0
    old = 3
    for i in range(len(val_a)):
        mine = mymatrix(combs[i])
        test = np.array([[1,0],[0,angle]])
        distance = dist(test, mine)
        if distance < old:
            winner = i
            old = distance
    return combs[winner]

def dustin_ez(k: int, tgates: int):
    combs = ["A","T"]
    val_a , val_b = [],[]
    for i in range(tgates-1):
        all1 = [i+"A" for i in combs]
        all2 = [i+"T" for i in combs]
        combs = all1 + all2
    for i in range(len(combs)):
        seq = combs[i]
        a = 0
        b = 1
        for i in seq:
            if i == "A":
                a,b = H(a,b)
                a,b = A(a,b)
            if i == "T":
                a,b = H(a,b)
                a,b = T(a,b)
        val_a.append(a)
        val_b.append(b)
    angle = np.exp((1j*2*np.pi)/(2**(k+3)))
    winner = 0
    old = 3
    for i in range(len(val_a)):
        mine = mymatrix(combs[i])
        test = np.array([[1,0],[0,angle]])
        distance = dist(test, mine)
        if distance < old:
            winner = i
            old = distance
    return combs[winner]

def mymatrix(seq):
    ok1 = np.matmul(unit,unit)
    for i in range(len(seq)):
        if seq[i] == "A":
            ok1 = np.matmul(ok1,h)
            ok1 = np.matmul(ok1,tdg)
        if seq[i] == "T":
            ok1 = np.matmul(ok1,h)
            ok1 = np.matmul(ok1,t)
        if seq[i] == "S":
            ok1 = np.matmul(ok1,h)
            ok1 = np.matmul(ok1,s)
        if seq[i] == "B":
            ok1 = np.matmul(ok1,h)
            ok1 = np.matmul(ok1,sdg)
    return ok1

def kpbruder(decomp: list):
    lel = unit
    for i in decomp:
        if i == "t":
            lel = np.matmul(lel, t)
        elif i == "tdg":
            lel = np.matmul(lel, tdg)
        elif i == "sdg":
            lel = np.matmul(lel, sdg)
        elif i == "s":
            lel = np.matmul(lel, s)
        elif i == "h":
            lel = np.matmul(lel, h)
    return lel

In [ ]:
# skibidi = dustin_ez(1,8)
# print(skibidi)
# ok1 = mymatrix(skibidi)

müde = kpbruder(decomp=decomp)

# x = [i+1 for i in range(15)]

# y = []
# for i in x:
#     skibidi = dustin_ez(1,i)
#     ok = mymatrix(skibidi)
#     bro = dist(test, ok)
#     y.append(bro)

# plt.plot(x,y)
# plt.yscale("log")

In [ ]:
print("Abstand zu Einheitsmatrix lol: ", dist(test, unit))
print("Abstand zu Solovay Kitaev Decomp: ", dist(test, müde))
#print("Abstand zu meins: ", dist(test, ok1))

Abstand zu Einheitsmatrix lol:  0.11095791726984708
Abstand zu Solovay Kitaev Decomp:  0.04365895185311492
